In [3]:
import numpy as np
import cvxpy as cp
import pandas as pd

df_X = pd.read_csv('Xsvm.csv',header=None)
df_Y = pd.read_csv('ysvm.csv',header=None)
X = np.array(df_X,dtype=np.float64)
Y = np.array(df_Y,dtype=np.float64)
print(X.shape,Y.shape)

(500, 2) (500, 1)


In [4]:
# Convex Optimization
a = cp.Variable(len(Y))
R1 = cp.matmul(cp.diag(a),Y)
R2 = cp.matmul(X.T,R1)
R4 = cp.norm(R2)**2
R4.shape

()

In [5]:
P1 = cp.sum(a)
Const1 = P1 - 0.5*R4
# Const1 = np.reshape(Const1,(1,))

In [6]:
Const2 = cp.matmul(a.T,Y)
Const3 = [0<=a,Const2 == 0]
obj = cp.Maximize(Const1)
prob = cp.Problem(obj, Const3)
prob.solve(verbose=True)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 2019, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 508, constraints m = 517
Cones:	primal zero / dual free vars: 8
	linear vars: 500
	soc vars: 3, soc blks: 1
	sd vars: 6, sd blks: 1
Setup time: 8.31e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.05e+20  1.11e+20  1.00e+00 -2.50e+21 -0.00e+00  1.88e+21  7.14e-02 
   100| 1.61e-01  9.62e-02  1.18e-02 -5.53e+01 -5.66e+01  1.19e-15  1.11e-01 
   200| 1.1

69.43750604453896

In [7]:
print(a.value)

[ 8.86070863e-08 -4.48732555e-08 -4.48258092e-08 -5.21008755e-08
  7.54486218e-08 -1.29804618e-08 -4.36789324e-08 -5.02640031e-08
 -8.03831859e-08 -7.40866316e-08 -2.74849213e-08 -5.44195947e-08
 -5.40945465e-08 -8.52215566e-08 -6.34182573e-08 -7.88261134e-08
 -5.24246001e-08 -2.58508417e-08 -3.29026935e-08 -3.30329322e-08
 -4.43318051e-08 -6.12721454e-08 -3.69605878e-08 -7.99112033e-08
 -7.12435827e-08 -6.22443503e-08 -3.92450328e-08 -5.71978827e-08
 -2.88586279e-08 -4.06109803e-08 -7.08467896e-08 -5.31405616e-08
 -4.76574698e-08  3.53512859e+00  6.29577105e-08 -6.09463034e-08
 -3.68561468e-08 -6.51746763e-08 -4.20466131e-08 -5.67267515e-08
 -9.41660752e-08 -5.59533272e-08 -6.83513902e-08 -4.54114390e-08
 -2.62101390e-08 -7.19594518e-08  2.24539919e-08 -6.25473061e-08
 -5.07928457e-08 -4.69014209e-08 -2.33179917e-08 -5.93522523e-08
 -6.76252933e-08 -2.33189636e-08 -9.21900342e-08 -5.90505699e-08
 -7.42432846e-08 -5.35735079e-08 -6.61689864e-08 -1.52111683e-08
 -6.18217062e-08 -6.63209

In [8]:
A = (np.array(a.value)).reshape(500,1)

In [9]:
W = np.zeros((2,))
for i in range(len(Y)):
    W += A[i]*Y[i]*(X[i].T)
    if(A[i]>1e-4):
        print(i)

33
281
469


In [10]:
print(W)

[-0.11845962 11.78393577]


In [11]:
W0 = (1/Y[281]) - np.dot(W,X[281])
print(W0)

[0.1023199]


In [12]:
Test = np.array([[2,0.5],[0.8,0.7],[1.58,1.33],[0.008, 0.001]])

for i in range(len(Test)):
    est = np.sign(np.dot(W,Test[i])+W0)
    print(Test[i],est)

[2.  0.5] [1.]
[0.8 0.7] [1.]
[1.58 1.33] [1.]
[0.008 0.001] [1.]


In [14]:
# Verification
from sklearn import svm

clf = svm.SVC()
clf.fit(X,Y)

/home/legion/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
clf.predict(Test)

array([1., 1., 1., 1.])